# Problema 1

O algoritmo estendido de Euclides (EXA) aceita dois inteiros constantes $$a, b > 0$$ e devolve inteiros $$r, s, t$$ tais que $$a \cdot s + b \cdot t = r$$ e $$r = \gcd(a, b)$$.

Para além das variáveis $$r, s, t$$, o código requer 3 variáveis adicionais $$r', s', t'$$ que representam os valores de $$r, s, t$$ no “próximo estado”.

**Algoritmo:**

```plaintext
INPUT a, b
assume a > 0 and b > 0
r, r', s, s', t, t' = a, b, 1, 0, 0, 1
while r' != 0
  q = r div r'
  r, r', s, s', t, t' = r', r − q × r', s', s − q × s', t', t − q × t'
OUTPUT r, s, t


### a) Construa um SFOTS usando BitVector’s de tamanho $$n$$ que descreva o comportamento deste programa.  Considere estado de erro quando $$\,r=0\,$$ ou alguma das variáveis atinge o “overflow”.

In [ ]:
from z3 import *

n = 5
BV = lambda x: BitVec(x, n)

a = BV("a")
b = BV("b")


r = BV("r")
r_linha = BV("rl")
s = BV("s")
s_linha = BV("sl")
t = BV("t")
t_linha = BV("tl")

r_next = BV("rn")
r_linha_next = BV("rln")
s_next = BV("sn")
s_linha_next = BV("sln")
t_next = BV("tn")
t_linha_next = BV("tln")

init = And(r == a, r_linha == b, s == 1, s_linha == 0, t == 0, t_linha == 1)

q = BV("q")
trans = And(
    q == r / r_linha,
    r_next == r_linha,
    r_linha_next == r - q * r_linha,
    s_next == s_linha,
    s_linha_next == s - q * s_linha,
    t_next == t_linha,
    t_linha_next == t - q * t_linha,
)

def checkOverflow_condition(rn, rln, sn, sln, tn, tln):
    condition = Or(
        rn < 0, rln < 0, sn < 0, sln < 0, tn < 0, tln < 0  
    )
    return Or(rn == 0, condition)

error = checkOverflow_condition(r_next, r_linha_next, s_next, s_linha_next, t_next, t_linha_next)

sfots = Solver()
sfots.add(init)
sfots.add(Not(error))
sfots.add(trans)

if sfots.check() == sat:
    model = sfots.model()

    r_value = model.eval(r)
    r_next_value = model.eval(r_next)
    r_linha_next_value = model.eval(r_linha_next)
    s_value = model.eval(s)
    s_next_value = model.eval(s_next)
    s_linha_next_value = model.eval(s_linha_next)
    t_value = model.eval(t)
    t_next_value = model.eval(t_next)
    t_linha_next_value = model.eval(t_linha_next)

    if r_value.as_long() == 0:
        print("Erro: A condição r == 0 foi admitida")
    else:
        print("Modelo Válido!")
        print(model)
    
    if checkOverflow_condition(r_next_value, r_linha_next_value, s_next_value, s_linha_next_value, t_next_value, t_linha_next_value):
        print("Erro: A condição de overflow foi admitida!")
    else: 
        print("Modelo Válido!")
        print(model)
else:
    print("Modelo Inválido!")


